# PosRecoCNN Training

Position reconstruction CNN training pipeline for SBND detector.
Uses ResNet-18 architecture to predict particle positions from PMT data.

## 1. Setup and Configuration

In [1]:
# Essential imports only
import os
import sys
import numpy as np
import awkward as ak
import uproot
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit

# Auto-detect training directory based on project structure
def find_training_directory():
    """Find the training directory automatically based on project structure."""
    
    # Method 1: Check if we're already in the training directory
    if os.path.exists('config.py') and os.path.exists('utils.py'):
        return os.getcwd()
    
    # Method 2: Look for sbndcode structure from common paths
    possible_bases = [
        # Standard LArSoft development paths
        "/exp/sbnd/app/users/{}/larsoft_*/srcs/sbndcode",
        "/exp/sbnd/app/users/{}/*/srcs/sbndcode", 
        # Alternative paths
        "/home/{}/sbndcode",
        "/home/{}/larsoft_*/srcs/sbndcode",
        # Current directory relatives
        "./sbndcode",
        "../sbndcode",
        "../../sbndcode",
    ]
    
    # Get current user
    import getpass
    username = getpass.getuser()
    
    # Expand paths with username and wildcards
    import glob
    expanded_paths = []
    for base in possible_bases:
        if '{}' in base:
            base = base.format(username)
        if '*' in base:
            expanded_paths.extend(glob.glob(base))
        else:
            expanded_paths.append(base)
    
    # Check each potential sbndcode directory
    for sbndcode_path in expanded_paths:
        training_path = os.path.join(sbndcode_path, "sbndcode", "PosRecoCVN", "training")
        if os.path.exists(training_path) and os.path.exists(os.path.join(training_path, "config.py")):
            return training_path
    
    # Method 3: Search upwards from current directory
    current = os.getcwd()
    for _ in range(5):  # Don't go too far up
        training_path = os.path.join(current, "sbndcode", "PosRecoCVN", "training")
        if os.path.exists(training_path) and os.path.exists(os.path.join(training_path, "config.py")):
            return training_path
        parent = os.path.dirname(current)
        if parent == current:  # reached root
            break
        current = parent
    
    return None

# Find and set training directory
print("🔍 Auto-detecting training directory...")
training_dir = find_training_directory()

if training_dir:
    print(f"✅ Found training directory: {training_dir}")
    if os.getcwd() != training_dir:
        print(f"📁 Changing directory from {os.getcwd()} to {training_dir}")
        os.chdir(training_dir)
else:
    print("❌ Could not find training directory automatically")
    print("🔧 Manual setup options:")
    print("1. Navigate to your training directory:")
    print("   os.chdir('/path/to/your/sbndcode/sbndcode/PosRecoCVN/training')")
    print("2. Or copy the required files:")
    print("   !cp /path/to/your/training/config.py .")
    print("   !cp /path/to/your/training/utils.py .")
    print("3. Or add your training directory to Python path:")
    print("   sys.path.append('/path/to/your/training/directory')")

# Add current directory to Python path
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

# Debug info
print(f"\nCurrent working directory: {os.getcwd()}")
print(f"config.py exists: {os.path.exists('config.py')}")  
print(f"utils.py exists: {os.path.exists('utils.py')}")

# TensorFlow
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Try to import local modules
config_imported = False
utils_imported = False

try:
    from config import *
    config_imported = True
    print("✅ config module imported successfully")
except ImportError as e:
    print(f"❌ config import error: {e}")

try:
    from utils import *
    utils_imported = True
    print("✅ utils module imported successfully")
except ImportError as e:
    print(f"❌ utils import error: {e}")

if not (config_imported and utils_imported):
    print("\n🔧 If auto-detection failed, try one of these solutions:")
    print("1. Manual directory change:")
    print("   os.chdir('/exp/sbnd/app/users/YOUR_USERNAME/larsoft_*/srcs/sbndcode/sbndcode/PosRecoCVN/training')")
    print("2. Add to Python path:")
    print("   sys.path.append('/path/to/your/training/directory')")
    print("3. Copy files to current directory")
    print("\nThen restart this cell.")

# TensorFlow settings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

# Only run these if modules were imported successfully
if config_imported and utils_imported:
    print("🎯 PosRecoCNN Training Pipeline")
    print_config_summary()
    validate_paths()
else:
    print("⚠️ Skipping configuration check until modules are properly imported")

🔍 Auto-detecting training directory...
✅ Found training directory: /exp/sbnd/app/users/svidales/larsoft_v10_06_01_develop/srcs/sbndcode/sbndcode/PosRecoCVN/training
📁 Changing directory from /home/svidales to /exp/sbnd/app/users/svidales/larsoft_v10_06_01_develop/srcs/sbndcode/sbndcode/PosRecoCVN/training

Current working directory: /exp/sbnd/app/users/svidales/larsoft_v10_06_01_develop/srcs/sbndcode/sbndcode/PosRecoCVN/training
config.py exists: True
utils.py exists: True


2025-08-04 16:06:03.650630: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-04 16:06:03.650721: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-04 16:06:03.652107: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-04 16:06:03.659629: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


✅ config module imported successfully
✅ utils module imported successfully
🎯 PosRecoCNN Training Pipeline
📋 Configuration Summary
PMT Maps Directory: /exp/sbnd/app/users/svidales/larsoft_v10_06_01_develop/srcs/sbndcode/sbndcode/PosRecoCVN/pmt_maps
Model Export Path: /exp/sbnd/data/users/svidales/AI_nuvT_project_support/cnn_models/
Training Data: mc_MCP2025B_02_prodgenie_corsika_proton_rockbox_sbnd_CV_reco2_sbnd_8k_test.root
Test Data: mc_MCP2025B_02_prodgenie_corsika_proton_rockbox_sbnd_CV_reco2_sbnd_1_test.root

Filter Settings:
  Energy cut: >50.0 MeV
  Position ranges: X(-200.0, 200.0), Y(-200.0, 200.0), Z(0.0, 500.0)

Training Settings:
  Epochs: 80
  Batch size: 16
  Train/Val/Test split: 90%/5%/5%
✅ All configuration paths validated successfully


## 2. Data Loading

In [2]:
# Load training data
print("📂 Loading training data...")
import time
start_time = time.time()

file = uproot.open(DATA_CONFIG['training_file'])
optree = file['opanatree']['OpAnaTree']

if LOADING_CONFIG['load_all']:
    print(f"Loading all events: {optree.num_entries:,}")
    arrays = []
    for i, key in enumerate(DATA_CONFIG['keys_to_load']):
        print(f"  Loading {key}... ({i+1}/{len(DATA_CONFIG['keys_to_load'])}) [{100*(i+1)/len(DATA_CONFIG['keys_to_load']):.1f}%]")
        array = optree[key].array()
        arrays.append(array)
else:
    n_total = optree.num_entries
    n_load = int(n_total * LOADING_CONFIG['fraction'])
    start = LOADING_CONFIG['start_event']
    end = start + n_load
    print(f"Loading events {start}-{end} ({n_load:,}/{n_total:,})")
    arrays = []
    for i, key in enumerate(DATA_CONFIG['keys_to_load']):
        print(f"  Loading {key}... ({i+1}/{len(DATA_CONFIG['keys_to_load'])}) [{100*(i+1)/len(DATA_CONFIG['keys_to_load']):.1f}%]")
        array = optree[key].array(entry_start=start, entry_stop=end)
        arrays.append(array)

loading_time = time.time() - start_time

# Unpack arrays
f_ophit_PE, f_ophit_ch, f_ophit_t, nuvT, dEpromx, dEpromy, dEpromz, dEtpc, nuvZ = arrays

# Display loading statistics
print("✅ Data loaded successfully")
print(f"⏱️ Loading time: {loading_time:.2f} seconds")
print(f"📊 Loaded {len(arrays)} arrays:")
for i, key in enumerate(DATA_CONFIG['keys_to_load']):
    array = arrays[i]
    array_size_mb = array.nbytes / (1024*1024)
    print(f"   {key}: {len(array):,} events, {array_size_mb:.1f} MB")

total_size_mb = sum(array.nbytes for array in arrays) / (1024*1024)
print(f"📈 Total data size: {total_size_mb:.1f} MB")
print(f"⚡ Loading speed: {total_size_mb/loading_time:.1f} MB/s")

📂 Loading training data...
Loading all events: 136,560
  Loading flash_ophit_pe... (1/9) [11.1%]
  Loading flash_ophit_ch... (2/9) [22.2%]
  Loading flash_ophit_time... (3/9) [33.3%]
  Loading nuvT... (4/9) [44.4%]
  Loading dEpromx... (5/9) [55.6%]
  Loading dEpromy... (6/9) [66.7%]
  Loading dEpromz... (7/9) [77.8%]
  Loading dEtpc... (8/9) [88.9%]
  Loading nuvZ... (9/9) [100.0%]
✅ Data loaded successfully
⏱️ Loading time: 735.79 seconds
📊 Loaded 9 arrays:
   flash_ophit_pe: 136,560 events, 2573.8 MB
   flash_ophit_ch: 136,560 events, 1288.8 MB
   flash_ophit_time: 136,560 events, 2573.8 MB
   nuvT: 136,560 events, 2.9 MB
   dEpromx: 136,560 events, 3.1 MB
   dEpromy: 136,560 events, 3.1 MB
   dEpromz: 136,560 events, 3.1 MB
   dEtpc: 136,560 events, 3.1 MB
   nuvZ: 136,560 events, 2.9 MB
📈 Total data size: 6454.8 MB
⚡ Loading speed: 8.8 MB/s


## 3. Channel Dictionary and Event Filtering

In [3]:
# Create channel dictionary
PDSMap = file['opanatree']['PDSMapTree']
ID = PDSMap['OpDetID'].array()
Type = PDSMap['OpDetType'].array()
channel_dict = {id_val: int(type_val) for id_val, type_val in zip(ID[0], Type[0])}
print(f"📡 Channel dictionary created: {len(channel_dict)} channels")

# Process events with centralized configuration
print("\n🔄 Processing events...")
results = process_events(
    nuvT, f_ophit_PE, f_ophit_ch, f_ophit_t,
    dEpromx, dEpromy, dEpromz, dEtpc, nuvZ,
    channel_dict, FILTER_CONFIG, verbose=True
)

# Unpack results
(nuvT_final, f_ophit_PE_final, f_ophit_ch_final, f_ophit_t_final,
 dEpromx_final, dEpromy_final, dEpromz_final, dEtpc_final, nuvZ_final, stats) = results

📡 Channel dictionary created: 312 channels

🔄 Processing events...
🚀 Starting optimized event processing...
Initial events: 136,560
Processing flashes...
Applying data validity cuts...
Applying energy cuts...
Applying position cuts...
✅ Processing completed in 1.94 seconds

Initial events: 136,560


,Cut,Removed,Remaining,Cumulative_Eff
0,Single neutrino,79136,57424,0.421
1,Has flashes,835,56589,0.414
2,Valid data (≠ -999 in dEprom),99,56490,0.414
3,Energy cut,2803,53687,0.393
4,Position cut,256,53431,0.391


Final efficiency: 0.391 (53,431/136,560)

📊 Final dataset ranges:
  dEpromx: [-200.0, 199.9]
  dEpromy: [-199.5, 199.5]
  dEpromz: [5.1, 500.0]
  dEtpc: [50.0, 10567.3]


## 4. PE Matrix and Image Creation

In [4]:
# Create PE matrix
print("🔢 Creating PE matrix...")
pe_matrix = create_pe_matrix(f_ophit_PE_final, f_ophit_ch_final, IMAGE_CONFIG['max_channels'])

# Load PMT maps with relative paths
print("\n🗺️ Loading PMT maps...")
uncoated_map = np.loadtxt(DATA_CONFIG['pmt_maps']['uncoated'], delimiter=",", dtype=int)
coated_map = np.loadtxt(DATA_CONFIG['pmt_maps']['coated'], delimiter=",", dtype=int)
print(f"Uncoated map shape: {uncoated_map.shape}")
print(f"Coated map shape: {coated_map.shape}")

# Create images and store normalization factors
print("\n🖼️ Creating PE images...")
images, normalization_factors = create_pe_images(
    pe_matrix, uncoated_map, coated_map, 
    method=IMAGE_CONFIG['selection_method']
)

print(f"✅ Images created: {images.shape}")
print(f"📊 Normalization factors: {normalization_factors}")

🔢 Creating PE matrix...
Creating PE matrix for 53,431 events x 312 channels
  Processing event 0/53,431 (0.0%)
  Processing event 10,000/53,431 (18.7%)
  Processing event 20,000/53,431 (37.4%)
  Processing event 30,000/53,431 (56.1%)
  Processing event 40,000/53,431 (74.9%)
  Processing event 50,000/53,431 (93.6%)
✅ Completed in 44.89 seconds
Matrix shape: (53431, 312)
Non-zero elements: 7,361,387
Total PE: 935784256.0

🗺️ Loading PMT maps...
Uncoated map shape: (118, 70)
Coated map shape: (118, 70)

🖼️ Creating PE images...
Creating 53,431 images (59×70×2)
[Norm] Shared max value for maps 0 & 1: 10789.310
✅ Images created: (53431, 59, 70, 2)
📊 Normalization factors: [10789.31]


## 5. Target Variable Preparation

In [5]:
# Prepare coordinates (use absolute x)
print("🎯 Preparing target variables...")
x_abs = np.abs(np.array(dEpromx_final).flatten())
y = np.array(dEpromy_final).flatten()
z = np.array(dEpromz_final).flatten()

coordinates = np.column_stack((x_abs, y, z))
print(f"Coordinate ranges before scaling:")
print(f"  X (abs): [{np.min(x_abs):.1f}, {np.max(x_abs):.1f}]")
print(f"  Y: [{np.min(y):.1f}, {np.max(y):.1f}]")
print(f"  Z: [{np.min(z):.1f}, {np.max(z):.1f}]")

# Scale coordinates using config
y_scaled = scale_coordinates(coordinates, COORD_CONFIG['ranges'])
print(f"\nScaled coordinate ranges:")
print(f"  X: [{np.min(y_scaled[:, 0]):.3f}, {np.max(y_scaled[:, 0]):.3f}]")
print(f"  Y: [{np.min(y_scaled[:, 1]):.3f}, {np.max(y_scaled[:, 1]):.3f}]")
print(f"  Z: [{np.min(y_scaled[:, 2]):.3f}, {np.max(y_scaled[:, 2]):.3f}]")

🎯 Preparing target variables...
Coordinate ranges before scaling:
  X (abs): [0.9, 200.0]
  Y: [-199.5, 199.5]
  Z: [5.1, 500.0]

Scaled coordinate ranges:
  X: [0.004, 1.000]
  Y: [-0.998, 0.998]
  Z: [0.010, 1.000]


## 6. Train/Validation/Test Split

In [6]:
# Calculate split sizes
n_total = len(y_scaled)
n_train = int(n_total * TRAINING_CONFIG['train_fraction'])
n_val = int(n_total * TRAINING_CONFIG['val_fraction'])
n_test = n_total - n_train - n_val

print(f"📊 Dataset splits:")
print(f"  Training: {n_train:,} ({100*n_train/n_total:.1f}%)")
print(f"  Validation: {n_val:,} ({100*n_val/n_total:.1f}%)")
print(f"  Test: {n_test:,} ({100*n_test/n_total:.1f}%)")

# Create splits
x_train = images[:n_train]
x_val = images[n_train:n_train+n_val]
x_test = images[-n_test:]

y_train = y_scaled[:n_train]
y_val = y_scaled[n_train:n_train+n_val]
y_test = y_scaled[-n_test:]

print(f"\n✅ Data splits created:")
print(f"  x_train: {x_train.shape}")
print(f"  x_val: {x_val.shape}")
print(f"  x_test: {x_test.shape}")

📊 Dataset splits:
  Training: 48,087 (90.0%)
  Validation: 2,671 (5.0%)
  Test: 2,673 (5.0%)

✅ Data splits created:
  x_train: (48087, 59, 70, 2)
  x_val: (2671, 59, 70, 2)
  x_test: (2673, 59, 70, 2)


## 7. Model Definition (ResNet-18 Architecture)

In [7]:
# Define ResNet-18 architecture (keeping original as it works well)
def create_resnet18_model(input_shape, dropout_rate=0.4):
    """Create ResNet-18 model for position regression."""
    
    # Input layer
    input_layer = layers.Input(shape=input_shape)

    # Initial convolution layer
    x = layers.Conv2D(64, kernel_size=7, strides=2, padding='same', use_bias=False)(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')(x)

    # Define the basic residual block
    def basic_block(input_tensor, filters, stride=1, downsample=False):
        shortcut = input_tensor

        x = layers.Conv2D(filters, kernel_size=3, strides=stride, padding='same', use_bias=False)(input_tensor)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)

        x = layers.Conv2D(filters, kernel_size=3, strides=1, padding='same', use_bias=False)(x)
        x = layers.BatchNormalization()(x)

        if downsample:
            shortcut = layers.Conv2D(filters, kernel_size=1, strides=stride, padding='same', use_bias=False)(input_tensor)
            shortcut = layers.BatchNormalization()(shortcut)

        x = layers.Add()([x, shortcut])
        x = layers.ReLU()(x)
        return x

    # ResNet-18 stages
    x = basic_block(x, 64)
    x = basic_block(x, 64)

    x = basic_block(x, 128, stride=2, downsample=True)
    x = basic_block(x, 128)

    x = basic_block(x, 256, stride=1, downsample=True)
    x = basic_block(x, 256)

    x = basic_block(x, 512, stride=1, downsample=True)
    x = basic_block(x, 512)

    # Global Average Pooling and output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(dropout_rate)(x)
    output = layers.Dense(3)(x)  # 3 coordinates: x, y, z

    return models.Model(inputs=input_layer, outputs=output)

# Create model
print("🏗️ Creating ResNet-18 model...")
input_shape = x_train.shape[1:]
model = create_resnet18_model(input_shape, TRAINING_CONFIG['dropout_rate'])

# Compile model
model.compile(
    optimizer=TRAINING_CONFIG['optimizer'],
    loss=TRAINING_CONFIG['loss'],
    metrics=TRAINING_CONFIG['metrics']
)

print(f"✅ Model created with input shape: {input_shape}")
print(f"📊 Total parameters: {model.count_params():,}")

🏗️ Creating ResNet-18 model...


2025-08-04 16:26:33.964697: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-08-04 16:26:34.026084: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-08-04 16:26:34.026497: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

✅ Model created with input shape: (59, 70, 2)
📊 Total parameters: 11,184,515


## 8. Model Training

In [8]:
# Configure TensorFlow GPU settings
print("🔧 Configuring TensorFlow...")

# Check GPU availability and configure memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Enable memory growth to avoid GPU memory allocation issues
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ Found {len(gpus)} GPU(s), memory growth enabled")
        
        # List GPU details
        for i, gpu in enumerate(gpus):
            print(f"  GPU {i}: {gpu}")
            
    except RuntimeError as e:
        print(f"⚠️ GPU configuration error: {e}")
        print("This is normal if GPU context was already initialized")
else:
    print("ℹ️ No GPUs found, using CPU")

# Verify data types and shapes before training
print(f"\n📊 Data verification before training:")
print(f"  x_train shape: {x_train.shape}, dtype: {x_train.dtype}")
print(f"  y_train shape: {y_train.shape}, dtype: {y_train.dtype}")
print(f"  x_val shape: {x_val.shape}, dtype: {x_val.dtype}")
print(f"  y_val shape: {y_val.shape}, dtype: {y_val.dtype}")

# Memory calculation
batch_size = TRAINING_CONFIG['batch_size']
input_size = np.prod(x_train.shape[1:]) * 4  # float32 = 4 bytes
batch_memory_mb = (batch_size * input_size) / (1024*1024)
print(f"  Estimated memory per batch: {batch_memory_mb:.1f} MB")

# Check for any NaN or infinite values
print(f"\n🔍 Data quality check:")
print(f"  x_train - NaN: {np.isnan(x_train).sum()}, Inf: {np.isinf(x_train).sum()}")
print(f"  y_train - NaN: {np.isnan(y_train).sum()}, Inf: {np.isinf(y_train).sum()}")

# Convert to float32 if needed (GPU prefers float32)
if x_train.dtype != np.float32:
    print("Converting x_train to float32...")
    x_train = x_train.astype(np.float32)
    x_val = x_val.astype(np.float32)
    x_test = x_test.astype(np.float32)

if y_train.dtype != np.float32:
    print("Converting y_train to float32...")
    y_train = y_train.astype(np.float32)
    y_val = y_val.astype(np.float32)
    y_test = y_test.astype(np.float32)

# Setup callbacks
print("\n⚙️ Setting up training callbacks...")
callbacks = [
    ModelCheckpoint(
        MODEL_CONFIG['weights_file'],
        monitor='val_loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=TRAINING_CONFIG['patience'],
        restore_best_weights=True
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=TRAINING_CONFIG['reduce_lr_factor'],
        patience=TRAINING_CONFIG['reduce_lr_patience'],
        min_lr=TRAINING_CONFIG['min_lr'],
        verbose=1
    )
]

# Try different strategies for GPU memory issues
print("\n🚀 Starting training...")

# Strategy 1: Try with original batch size
try:
    print(f"Attempting training with batch size {TRAINING_CONFIG['batch_size']}")
    history = model.fit(
        x_train, y_train,
        epochs=TRAINING_CONFIG['epochs'],
        batch_size=TRAINING_CONFIG['batch_size'],
        callbacks=callbacks,
        validation_data=(x_val, y_val),
        verbose=1
    )
    
    # Load best weights
    model.load_weights(MODEL_CONFIG['weights_file'])
    print("\n✅ Training completed and best weights loaded")
    
except Exception as e:
    print(f"❌ Training failed with batch size {TRAINING_CONFIG['batch_size']}: {e}")
    
    # Strategy 2: Try with smaller batch size
    smaller_batch_size = max(1, TRAINING_CONFIG['batch_size'] // 2)
    print(f"\n🔧 Trying with smaller batch size: {smaller_batch_size}")
    
    try:
        history = model.fit(
            x_train, y_train,
            epochs=TRAINING_CONFIG['epochs'],
            batch_size=smaller_batch_size,
            callbacks=callbacks,
            validation_data=(x_val, y_val),
            verbose=1
        )
        
        # Load best weights
        model.load_weights(MODEL_CONFIG['weights_file'])
        print(f"\n✅ Training completed with batch size {smaller_batch_size} and best weights loaded")
        
    except Exception as e2:
        print(f"❌ Training also failed with smaller batch size: {e2}")
        
        # Strategy 3: Try with much smaller batch size
        tiny_batch_size = max(1, TRAINING_CONFIG['batch_size'] // 4)
        print(f"\n🔧 Final attempt with tiny batch size: {tiny_batch_size}")
        
        history = model.fit(
            x_train, y_train,
            epochs=TRAINING_CONFIG['epochs'],
            batch_size=tiny_batch_size,
            callbacks=callbacks,
            validation_data=(x_val, y_val),
            verbose=1
        )
        
        # Load best weights
        model.load_weights(MODEL_CONFIG['weights_file'])
        print(f"\n✅ Training completed with batch size {tiny_batch_size} and best weights loaded")

🔧 Configuring TensorFlow...
⚠️ GPU configuration error: Physical devices cannot be modified after being initialized
This is normal if GPU context was already initialized

📊 Data verification before training:
  x_train shape: (48087, 59, 70, 2), dtype: float32
  y_train shape: (48087, 3), dtype: float64
  x_val shape: (2671, 59, 70, 2), dtype: float32
  y_val shape: (2671, 3), dtype: float64
  Estimated memory per batch: 0.5 MB

🔍 Data quality check:
  x_train - NaN: 0, Inf: 0
  y_train - NaN: 0, Inf: 0
Converting y_train to float32...

⚙️ Setting up training callbacks...

🚀 Starting training...
Attempting training with batch size 16
❌ Training failed with batch size 16: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

🔧 Trying with smaller batch size: 8


2025-08-04 16:29:23.529659: W external/local_tsl/tsl/framework/bfc_allocator.cc:487] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.48GiB (rounded to 1588794624)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-08-04 16:29:23.529819: I external/local_tsl/tsl/framework/bfc_allocator.cc:1044] BFCAllocator dump for GPU_0_bfc
2025-08-04 16:29:23.529847: I external/local_tsl/tsl/framework/bfc_allocator.cc:1051] Bin (256): 	Total Chunks: 56, Chunks in use: 56. 14.0KiB allocated for chunks. 14.0KiB in use in bin. 5.2KiB client-requested in use in bin.
2025-08-04 16:29:23.529860: I external/local_tsl/tsl/framework/bfc_allocator.cc:1051] Bin (512): 	Total Chunks: 20, Chunks in use: 20. 10.0KiB allocated for chunks. 10.0KiB in use in bin. 10.0KiB client-requested in use in bin.
2025-08-04 16:29:2

❌ Training also failed with smaller batch size: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

🔧 Final attempt with tiny batch size: 4


2025-08-04 16:29:48.731780: W external/local_tsl/tsl/framework/bfc_allocator.cc:487] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.48GiB (rounded to 1588794624)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-08-04 16:29:48.731926: I external/local_tsl/tsl/framework/bfc_allocator.cc:1044] BFCAllocator dump for GPU_0_bfc
2025-08-04 16:29:48.731953: I external/local_tsl/tsl/framework/bfc_allocator.cc:1051] Bin (256): 	Total Chunks: 56, Chunks in use: 56. 14.0KiB allocated for chunks. 14.0KiB in use in bin. 5.2KiB client-requested in use in bin.
2025-08-04 16:29:48.731967: I external/local_tsl/tsl/framework/bfc_allocator.cc:1051] Bin (512): 	Total Chunks: 20, Chunks in use: 20. 10.0KiB allocated for chunks. 10.0KiB in use in bin. 10.0KiB client-requested in use in bin.
2025-08-04 16:29:4

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

## 9. Training History Visualization

In [ ]:
# Plot training history
plt.figure(figsize=(12, 5))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss', color='royalblue', linewidth=2)
plt.plot(history.history['val_loss'], label='Validation Loss', color='darkorange', linewidth=2)
plt.yscale('log')
plt.xlabel('Epochs')
plt.ylabel('Loss (log scale)')
plt.title('Model Loss')
plt.legend()
plt.grid(True, alpha=0.3)

# MSE plot
plt.subplot(1, 2, 2)
plt.plot(history.history['mse'], label='Training MSE', color='royalblue', linewidth=2)
plt.plot(history.history['val_mse'], label='Validation MSE', color='darkorange', linewidth=2)
plt.yscale('log')
plt.xlabel('Epochs')
plt.ylabel('MSE (log scale)')
plt.title('Model MSE')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print final metrics
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]
print(f"📊 Final Training Loss: {final_train_loss:.6f}")
print(f"📊 Final Validation Loss: {final_val_loss:.6f}")

## 10. Model Export

In [ ]:
# Export model with automatic naming
export_path = get_model_export_path(len(y_train))
print(f"💾 Exporting model to: {export_path}")

model.export(export_path)
print("✅ Model exported successfully")

# Save normalization factors for inference
import json
norm_file = os.path.join(os.path.dirname(export_path), 'normalization_factors.json')
with open(norm_file, 'w') as f:
    json.dump({
        'normalization_factors': normalization_factors,
        'coord_ranges': COORD_CONFIG['ranges'],
        'image_config': IMAGE_CONFIG
    }, f, indent=2)
    
print(f"💾 Normalization factors saved to: {norm_file}")

## 11. Model Evaluation on Test Set

In [ ]:
# Make predictions on test set
print("🔮 Making predictions on test set...")
y_pred_scaled = model.predict(x_test)

# Convert back to original coordinates
y_pred_original = inverse_scale_coordinates(y_pred_scaled, COORD_CONFIG['ranges'])
y_test_original = inverse_scale_coordinates(y_test, COORD_CONFIG['ranges'])

print(f"✅ Predictions completed for {len(y_test):,} test events")
print(f"Prediction ranges:")
print(f"  X: [{np.min(y_pred_original[:, 0]):.1f}, {np.max(y_pred_original[:, 0]):.1f}]")
print(f"  Y: [{np.min(y_pred_original[:, 1]):.1f}, {np.max(y_pred_original[:, 1]):.1f}]")
print(f"  Z: [{np.min(y_pred_original[:, 2]):.1f}, {np.max(y_pred_original[:, 2]):.1f}]")

## 12. Performance Analysis

In [ ]:
# Reco vs Truth comparison plots
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
coord_names = ['X', 'Y', 'Z']
coord_limits = [(0, 200), (-200, 200), (0, 500)]

for i, (coord, limits) in enumerate(zip(coord_names, coord_limits)):
    hist, xedges, yedges = np.histogram2d(
        y_test_original[:, i],
        y_pred_original[:, i],
        bins=ANALYSIS_CONFIG['reco_truth_comparison']['bins']
    )
    
    hist_masked = np.ma.masked_equal(hist, 0)
    
    im = axs[i].pcolormesh(
        xedges, yedges, hist_masked.T,
        cmap=ANALYSIS_CONFIG['reco_truth_comparison']['cmap'],
        shading='auto'
    )
    
    axs[i].plot(limits, limits, 'r-', linewidth=2, label='Perfect prediction')
    axs[i].set_xlim(limits)
    axs[i].set_ylim(limits)
    axs[i].set_xlabel(f'Truth {coord} [cm]')
    axs[i].set_ylabel(f'Reco {coord} [cm]')
    axs[i].set_title(f'{coord} coordinate')
    axs[i].grid(True, alpha=0.3)
    axs[i].legend()

plt.tight_layout()
plt.show()

# Calculate and display performance metrics
mse_per_coord = np.mean((y_pred_original - y_test_original)**2, axis=0)
rmse_per_coord = np.sqrt(mse_per_coord)
mae_per_coord = np.mean(np.abs(y_pred_original - y_test_original), axis=0)

print("\n📊 Performance Metrics:")
print("=" * 40)
for i, coord in enumerate(coord_names):
    print(f"{coord} coordinate:")
    print(f"  RMSE: {rmse_per_coord[i]:.2f} cm")
    print(f"  MAE:  {mae_per_coord[i]:.2f} cm")
    print()

overall_rmse = np.sqrt(np.mean(mse_per_coord))
print(f"Overall RMSE: {overall_rmse:.2f} cm")

## 13. Bias Analysis

In [ ]:
# Calculate differences (bias)
diff_coords = y_pred_original - y_test_original

# Gaussian fit function
def gaussian(x, a, x0, sigma):
    return a * np.exp(-(x - x0)**2 / (2 * sigma**2))

# Bias analysis plots
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
bias_results = {}

for i, coord in enumerate(coord_names):
    diff = diff_coords[:, i]
    config = ANALYSIS_CONFIG['bias_analysis']
    
    # Create histogram
    counts, bins, _ = axs[i].hist(
        diff, 
        config['hist_bins'], 
        range=config['hist_ranges'][coord],
        alpha=0.7,
        label='Data'
    )
    
    # Prepare for Gaussian fit
    bin_centers = (bins[:-1] + bins[1:]) / 2
    fit_range = config['fit_ranges'][coord]
    
    # Fit Gaussian
    try:
        p0 = [np.max(counts), 0, 50]
        popt, _ = curve_fit(
            gaussian, 
            bin_centers[fit_range[0]:fit_range[1]], 
            counts[fit_range[0]:fit_range[1]], 
            p0=p0
        )
        
        # Plot fit
        x_fit = np.linspace(bin_centers[fit_range[0]], bin_centers[fit_range[1]], 100)
        y_fit = gaussian(x_fit, *popt)
        axs[i].plot(x_fit, y_fit, 'r-', linewidth=2, label='Gaussian fit')
        
        # Store results
        bias_results[coord] = {
            'mean': popt[1],
            'sigma': popt[2]
        }
        
    except Exception as e:
        print(f"Warning: Could not fit Gaussian for {coord}: {e}")
        bias_results[coord] = {
            'mean': np.mean(diff),
            'sigma': np.std(diff)
        }
    
    axs[i].axvline(0, color='black', linestyle='--', alpha=0.5, label='Zero bias')
    axs[i].set_xlabel(f'Bias {coord} [cm]')
    axs[i].set_ylabel('Count')
    axs[i].set_title(f'{coord} coordinate bias')
    axs[i].legend()
    axs[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print bias results
print("\n📊 Bias Analysis Results:")
print("=" * 40)
for coord in coord_names:
    mean = bias_results[coord]['mean']
    sigma = bias_results[coord]['sigma']
    print(f"{coord} coordinate:")
    print(f"  Mean bias: {mean:.2f} cm")
    print(f"  Resolution (σ): {sigma:.2f} cm")
    print()

## 14. Summary and Next Steps

In [ ]:
print("🎉 Training Pipeline Completed Successfully!")
print("=" * 50)
print(f"📊 Dataset: {len(y_train):,} training events")
print(f"🏗️ Model: ResNet-18 architecture")
print(f"💾 Model saved to: {export_path}")
print(f"📈 Final validation loss: {final_val_loss:.6f}")
print(f"📏 Overall RMSE: {overall_rmse:.2f} cm")

print("\n📋 For inference on new data:")
print("1. Use the saved normalization factors")
print("2. Apply same image creation pipeline")
print("3. Load exported model for predictions")
print("4. Apply inverse coordinate scaling")

print(f"\n📁 Files created:")
print(f"  - Model: {export_path}")
print(f"  - Normalization factors: {norm_file}")
print(f"  - Best weights: {MODEL_CONFIG['weights_file']}")